In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib
from keras_tuner.tuners import Hyperband
import matplotlib.pyplot as plt
from keras_tuner import HyperModel
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
from CleaveClassifier import *

In [ ]:
class MLPDataCollector(DataCollector):

    def __init__(self, csv_path, img_folder):
        super().__init__(csv_path, img_folder)
        

    def extract_data(self, feature_scaler_path=None, tension_scaler_path=None):
        '''
        Extract data from dataframe into separate lists for creating datasets.

        Parameters:
        ------------------------------------

        scalar_filename: str
        - path to store pickled scaler 

        Returns: list, list, list
        - lists of images, features, and labels
        '''
        images = self.df['ImagePath'].values
        #features = self.df[['CleaveAngle', 'CleaveTension']].values
        features = self.df[['CleaveAngle', 'ScribeDiameter', 'Misting', 'Hackle', 'Tearing']].values.astype(np.float32)
        labels = self.df['CleaveTension'].values.astype(np.float32)
        tension_scaler = MinMaxScaler()
        labels = tension_scaler.fit_transform(labels.reshape(-1, 1))
        feature_scaler = MinMaxScaler()
        features = feature_scaler.fit_transform(features)
        if feature_scaler_path:
            joblib.dump(self.scaler, f'{feature_scaler_path}.pkl')
        if tension_scaler_path:
            joblib.dump(self.scaler, f'{tension_scaler_path}.pkl')
        return images, features, labels
    
    def create_datasets(self, images, features, labels, test_size, buffer_size, batch_size):
        '''
        Creates test and train datasets and splits into different batches after shuffling.

        Parameters:
        -----------------------------------------

        images: list
        - paths to images in google drive
        features: list
        - numerical parameters to label images
        labels: int
        - targets to qualify image quality
        test_size: float
        - decimal between 0 and 1 to represent test size of dataset
        buffer_size: int
        - size of buffer for shuffling data
        batch_size: int
        - size to group data into

        Returns: tf.tensor
        - train and test datasets
        '''
        train_imgs, test_imgs, train_features, test_features, train_labels, test_labels = train_test_split(
            images, features, labels, test_size=test_size)
        train_ds = tf.data.Dataset.from_tensor_slices(((train_imgs, train_features), train_labels))
        test_ds = tf.data.Dataset.from_tensor_slices(((test_imgs, test_features), test_labels))

        # Map using bound method
        train_ds = train_ds.map(lambda x, y: self.process_images_features(x, y))
        test_ds = test_ds.map(lambda x, y: self.process_images_features(x, y))

        train_ds = train_ds.shuffle(buffer_size=buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        return train_ds, test_ds

In [ ]:
class BuildMLPModel(CustomModel):

    def __init__(self, cnn_model_path, train_ds, test_ds):
        super().__init__(train_ds, test_ds)
        self.cnn_model = tf.keras.models.load_model(cnn_model_path)
        self.image_input = self.cnn_model.input[0]
        self.feature_output = self.cnn_model.get_layer('dropout').output
       

    def build_pretrained_model(self, param_shape):
        '''
        Build model

        Returns:
        tf.keras.Model
            - Model to be trained
            
        '''
        # Pre-trained base model
        x = Dense(64, activation='relu')(self.feature_output)
        x = Dense(32, activation='relu')(x)
        feature_input = Input(shape=param_shape, name='feature_input')  # Features
        #angle_input = Input(shape=(1,), name='angle_input')  # New input
        y = Dense(16, activation='relu')(feature_input)
        #y = Dense(16, activation='relu')(angle_input

        combined = Concatenate()([x, y])
        z = Dense(64, activation='relu')(combined)
        output = Dense(1, name='tension_output')(z)
        # Use angle input for 250LA
        regression_model = Model(inputs=[self.image_input, feature_input], outputs=output)
        regression_model.summary()
        return regression_model
    
    def compile_model(self, param_shape, learning_rate=0.001):
      '''
      Compile model after calling build_model function

      Parameters:
      -------------------------------------
      image_shape: tuple
          - dimensions of images
      param_shape: tuple
          - dimensions of parameters
      learning_rate: float
          - learning rate for training model

      Returns:
      tf.keras.Model
          - Mode to be trained
      '''
      # Adaptive Moment Estimation optimizer
      # Set learning rate and then compile model
      # Loss functions is mean squared error for regression
      model = self.build_pretrained_model(param_shape)
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
      model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
      return model
    
    def create_early_stopping(self, patience=3, mode='min', monitor="val_mae"):
      '''
      Create early stopping callback to monitor training success and prevent overfitting.

      Parameters:
      ----------------------------------------

      patience: int
        - number of epochs to stop when monitor plateus
        - default: 3
      mode: str
        - max, min, avg
        - method to track monitor
        - default: max
      monitor: str
        - metric to monitor during training
        - default: val_accuracy
      
      Returns: tf.callbacks.EarlyStopping
        - early stopping callback
      '''
      es_callback = tf.keras.callbacks.EarlyStopping(
        monitor=monitor,
        patience=patience,
        mode = mode,
        restore_best_weights=True
      )
      return es_callback
    
    def create_checkpoints(self, checkpoint_filepath="/content/drive/MyDrive/mlp_checkpoints.keras", monitor="val_mae", mode="min", save_best_only=True):
      '''
      Create model checkpoints to avoid losing data while training

      Parameters:
      --------------------------------------

      checkpoint_filepath: str
        - path to save model checkpoints
        - default: /content/drive/MyDrive/checkpoints.keras
      monitor: str
        - metric to monitor during training
        - deafault: val_accuracy
      mode: str
        - max, min, avg
        - method to determine stoppping point of metric
        - default: max
      save_best_only: boolean
        - to determine if only best model shold be saved
        - deafault: True

      Returns: tf.callback.ModelCheckpoint
        - checkpoint to use during training
      '''
      model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_filepath,
        monitor=monitor,
        mode=mode,
        save_best_only=save_best_only,
        verbose=1
      )
      return model_checkpoint_callback

In [ ]:
class TensionPredictor:

    def __init__(self, model, image_folder, image_path, tension_scaler_path, feature_scaler_path):
        self.model = model
        self.image_path = image_path
        self.image_folder = image_folder
        self.tenion_scaler = joblib.load(tension_scaler_path)
        self.feature_scaler = joblib.load(feature_scaler_path)

    def load_and_preprocess_image(self, file_path, img_folder):
        '''
        Load and preprocess image from file path

        Parameters:
        -------------------------------------
        file_path: str
            - path to image file
        img_folder: str
            - path to image folder
            Returns:
        tf.Tensor
            - preprocessed image
        '''
        # Construct full path
        full_path = os.path.join(img_folder, file_path)
        img_raw = tf.io.read_file(full_path)
        img = tf.image.decode_png(img_raw, channels=1)
        img = tf.image.resize(img, [224, 224])
        img = tf.image.grayscale_to_rgb(img)
        # Normalize image
        img = img / 255.0
        return img

    def PredictTension(self, features):
        '''
        Predict tension for given image and angle

        Parameters:
        -------------------------------------
        model: tf.keras.Model
            - Model to be used for prediction
        image_path: str
            - Path to image to be used for prediction
            angle: float
            - Angle to be used for prediction

        Returns:
        float
            - Predicted tension
        '''
        # Process image and convert angle and image to tensor with dimension for single batch
        image = self.load_and_preprocess_image(self.image_folder, self.image_path)
        image = tf.expand_dims(image, axis=0)
        features = tf.convert_to_tensor(features.values, dtype=tf.float32)
        # Predict tension
        features = self.feature_scaler.transform(features)
        predicted_tension = self.model.predict([image, features])
        # Scale tension back to normal units
        predicted_tension = self.tension_scaler.inverse_transform(predicted_tension)
        # Print tensions
        return predicted_tension[0][0]

    def plot_metric(self, title, X, y, x_label, y_label, x_legend, y_legend):
        '''
        Plot metric

        Parameters:
        -------------------------------------
        title: str
            - Title of plot
        X: list
            - List of x values
        y: list
            - List of y values
            x_label: str
            - Label for x axis
        y_label: str
            - Label for y axis
        x_legend: str
            - Legend for x axis
        y_legend: str
            - Legend for y axis
        '''
        plt.title(title)
        plt.plot(X, label=x_legend)
        plt.plot(y, label=y_legend)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.legend(loc='lower right')
        plt.show()

In [ ]:
class BuildMLPHyperModel(HyperModel):
    '''
    This class build a HyperModel to determine optimal hyperparmeters
    '''
    def __init__(self, model_path):
        '''
        Parameters:
        -------------------------------------
        model: tf.keras.Model
            - Model to be used for hyperparameter tuning
        '''
        self.cnn_model = tf.keras.models.load_model(model_path)
        self.image_input = self.cnn_model.input[0]
        self.feature_output = self.cnn_model.get_layer('dropout').output

    def build(self, hp):
      '''
      Build model with hyperparameters

      Parameters:
      -------------------------------------
      hp: keras_tuner.HyperParameters
          - Hyperparameters to be used for tuning
      Returns:
      tf.keras.Model
          - Model to be trained
      '''
        # Pre-trained base model

      x = Dense(
            hp.Int('dense_param1', min_value=16, max_value=128, step=16),
            activation='relu')(self.feature_output)
      x = Dense(
            hp.Int('dense_param2', min_value=8, max_value=64, step=8),
            activation='relu')(x)

      feature_input = Input(shape=(5,), name='feature_input')  # Features
        #angle_input = Input(shape=(1,), name='angle_input')  # New input
      y = Dense(
            hp.Int('dense_angle', min_value=16, max_value=128, step=16),
            activation='relu')(feature_input)

      combined = Concatenate()([x, y])
      z = Dense(
            hp.Int('dense_combined', min_value=16, max_value=128, step=16),
            activation='relu')(combined)
      z = Dense(1)(z)

      mlp_hypermodel = Model(inputs=[self.image_input, feature_input], outputs=z)
      mlp_hypermodel.summary()

      mlp_hypermodel.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Choice('learning_rate', values=[0.0005, 0.001, 0.01])
            ),
            loss='mse',
            metrics=['mae']
        )

      return mlp_hypermodel

In [ ]:
class MLPHyperparameterTuning(HyperParameterTuning):

    def __init__(self, cnn_path, max_epochs=20, objective='val_mae', directory='/content/drive/MyDrive/Thorlabs', project_name='MLPTuner'):
      '''
      Parameters:
      -------------------------------------
      model: tf.keras.Model
          - Model to be used for hyperparameter tuning
      '''
      self.cnn_model = tf.keras.models.load_model(cnn_path)
      self.image_input = self.cnn_model.input[0]
      self.feature_output = self.cnn_model.get_layer('dropout').output
      hypermodel = BuildMLPHyperModel(cnn_path)
      self.tuner = Hyperband(
        hypermodel,
        objective=objective,
        max_epochs=max_epochs,
        directory=directory,
        project_name=project_name
    )